# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [81]:
import pandas as pd
house_sales = pd.read_csv("house_sales.csv")
#print(house_sales.head(10))
house_sales['city'] = house_sales['city'].astype('category')
missing_city = house_sales['city'].isnull().sum()
missing_city += (house_sales['city'] == '--').sum()
print(house_sales['city'].values)
print(missing_city)

['Silvertown', 'Silvertown', 'Riverford', 'Silvertown', 'Silvertown', ..., 'Riverford', 'Poppleton', 'Riverford', 'Teasdale', 'Poppleton']
Length: 1500
Categories (5, object): ['--', 'Poppleton', 'Riverford', 'Silvertown', 'Teasdale']
73


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [82]:
import pandas as pd

def data_treatment(df):
    # Cleaning city column
    df['city'].fillna('Unknown', inplace=True)
    df['city'] = df['city'].astype('category')
    # Lista de cidades válidas
    valid_cities = ['Poppleton', 'Riverford', 'Silvertown', 'Teasdale']

    # Mapeamento para substituir valores inválidos por 'Unknown'
    mapping = {city: city if city in valid_cities else 'Unknown' for city in df['city']}

    # Substituir valores na coluna 'city'
    df['city'] = df['city'].map(mapping)
    
    # Cleaning sale_price column
    df.dropna(subset=['sale_price'], inplace=True)
    df['sale_price'] = df['sale_price'].astype(int)
    
    # Cleaning sale_date column
    df['sale_date'].fillna('2023-01-01', inplace=True)
    # Convert 'sale_date' column to datetime type
    df['sale_date'] = pd.to_datetime(df['sale_date'])
    
    # Cleaning months_listed column
    mean_months_listed = df['months_listed'].mean()
    df['months_listed'].fillna(round(mean_months_listed, 1), inplace=True)
    
    # Cleaning bedrooms column
    mean_bedrooms = df['bedrooms'].mean()
    df['bedrooms'].fillna(round(mean_bedrooms, 0), inplace=True)
    df['bedrooms'] = df['bedrooms'].astype(int)
    
    # Cleaning house_type column
    categories_house_type = ['Terraced', 'Semi-detached', 'Detached']
    df['house_type'] = pd.Categorical(df['house_type'], categories=categories_house_type, ordered=True)
    most_common_house_type = df['house_type'].mode()[0]
    df['house_type'].fillna(most_common_house_type, inplace=True)
    
    # Cleaning area column
    df['area'] = df['area'].str.replace(' sq.m.', '').astype(float)
    mean_area = df['area'].mean()
    df['area'].fillna(round(mean_area, 1), inplace=True)
    
    return df
    
house_sales = pd.read_csv('house_sales.csv')
clean_data = data_treatment(house_sales)
print(clean_data['area'])

0       107.8
1       498.8
2       542.5
3       528.4
4       477.1
        ...  
1495    432.2
1496    599.8
1497    359.1
1498    253.7
1499    268.7
Name: area, Length: 1500, dtype: float64


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [83]:
import pandas as pd
house_sales = pd.read_csv("house_sales.csv")

# Convert 'sale_price' to numeric
house_sales['sale_price'] = pd.to_numeric(house_sales['sale_price'], errors='coerce')

# Calculate average sale price and variance by number of bedrooms
price_by_rooms = house_sales.groupby('bedrooms')['sale_price'].agg(['mean', 'var']).reset_index()
price_by_rooms = price_by_rooms.rename(columns={'mean': 'avg_price', 'var': 'var_price'})

# Round the values to one decimal place
price_by_rooms['avg_price'] = price_by_rooms['avg_price'].round(1)
price_by_rooms['var_price'] = price_by_rooms['var_price'].round(1)

# Display the resulting table
print(price_by_rooms)

   bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [84]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE

# Load the training and validation datasets
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('validation.csv')

def data_treatment(df):
    # Cleaning house_id column
    df.dropna(subset=['house_id'],inplace=True)
    #print(df['house_id'].is_unique)
    
    # Cleaning city column
    df['city'].fillna('Unknown',inplace=True)
    df['city'] = df['city'].astype('category')
    
    # Cleaning sale_date column
    df['sale_date'].fillna('2023-01-01',inplace=True)
    
    # Cleaning months_listed column
    mean_months_listed = df['months_listed'].mean()
    df['months_listed'].fillna(round(mean_months_listed,1),inplace=True)
    
    # Cleaning bedrooms column
    mean_bedrooms = df['bedrooms'].mean()
    df['bedrooms'].fillna(round(mean_bedrooms,0),inplace=True)
    
    # Cleaning house_type column
    categories_house_type = ['Terraced', 'Semi-detached', 'Detached']
    
    df['house_type'] = pd.Categorical(df['house_type'], categories=categories_house_type, ordered=True)
    most_common_house_type = df['house_type'].mode()[0]
    df['house_type'].fillna(most_common_house_type, inplace=True)
    
    # Cleaning area column
    #df['area'] = df['area'].str.replace(' sq.m.', '').astype(float)
    mean_area = df['area'].mean()
    df['area'].fillna(round(mean_area, 1), inplace=True)
    
    return df

train_data = data_treatment(train_data) 
val_data = data_treatment(val_data)

# Fit a Linear Regression model
model = LinearRegression()

# Encode 'house_type' and 'city' columns
train_data = pd.get_dummies(train_data, columns=['house_type', 'city'])
val_data = pd.get_dummies(val_data, columns=['house_type', 'city'])

# Convert to datetime to get year, month, day
train_data['sale_date'] = pd.to_datetime(train_data['sale_date'])
val_data['sale_date'] = pd.to_datetime(val_data['sale_date'])


# Extract useful date-related features in train_data
train_data['sale_year'] = train_data['sale_date'].dt.year
train_data['sale_month'] = train_data['sale_date'].dt.month
train_data['sale_day'] = train_data['sale_date'].dt.day

# Repeat the same process for val_data
val_data['sale_date'] = pd.to_datetime(val_data['sale_date'])
val_data['sale_year'] = val_data['sale_date'].dt.year
val_data['sale_month'] = val_data['sale_date'].dt.month
val_data['sale_day'] = val_data['sale_date'].dt.day

# Drop the original 'sale_date' columns
train_data.drop('sale_date', axis=1, inplace=True)
val_data.drop('sale_date', axis=1, inplace=True)

# Fit model, separating features and targets
model.fit(train_data.drop('sale_price', axis=1), train_data['sale_price'])

# Make predictions on the validation data
predicted_prices = model.predict(val_data)

# Create the base_result dataframe with house_id and predicted price
base_result = pd.DataFrame({'house_id': val_data['house_id'], 'price': predicted_prices})

# Display the resulting dataframe
print(base_result)


     house_id          price
0     1331375  121709.813918
1     1630115  304232.551410
2     1645745  384418.334267
3     1336775  125014.708890
4     1888274  270695.083194
..        ...            ...
295   1986255  349092.841750
296   1896276  368686.764539
297   1758223  257195.388196
298   1752010  169083.390810
299   1651404  388790.116582

[300 rows x 2 columns]


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [85]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Load the training and validation datasets
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('validation.csv')

def data_treatment(df):
    # Cleaning house_id column
    df.dropna(subset=['house_id'],inplace=True)
    #print(df['house_id'].is_unique)
    
    # Cleaning city column
    df['city'].fillna('Unknown',inplace=True)
    df['city'] = df['city'].astype('category')
    
    # Cleaning sale_date column
    df['sale_date'].fillna('2023-01-01',inplace=True)
    
    # Cleaning months_listed column
    mean_months_listed = df['months_listed'].mean()
    df['months_listed'].fillna(round(mean_months_listed,1),inplace=True)
    
    # Cleaning bedrooms column
    mean_bedrooms = df['bedrooms'].mean()
    df['bedrooms'].fillna(round(mean_bedrooms,0),inplace=True)
    
    # Cleaning house_type column
    categories_house_type = ['Terraced', 'Semi-detached', 'Detached']
    
    df['house_type'] = pd.Categorical(df['house_type'], categories=categories_house_type, ordered=True)
    most_common_house_type = df['house_type'].mode()[0]
    df['house_type'].fillna(most_common_house_type, inplace=True)
    
    # Cleaning area column
    #df['area'] = df['area'].str.replace(' sq.m.', '').astype(float)
    mean_area = df['area'].mean()
    df['area'].fillna(round(mean_area, 1), inplace=True)
    
    return df

train_data = data_treatment(train_data) 
val_data = data_treatment(val_data)

# Fit a Linear Regression model
model = RandomForestRegressor()

# Encode 'house_type' and 'city' columns
train_data = pd.get_dummies(train_data, columns=['house_type', 'city'])
val_data = pd.get_dummies(val_data, columns=['house_type', 'city'])


# Convert to datetime to get year, month, day
train_data['sale_date'] = pd.to_datetime(train_data['sale_date'])
val_data['sale_date'] = pd.to_datetime(val_data['sale_date'])


# Extract useful date-related features in train_data
train_data['sale_year'] = train_data['sale_date'].dt.year
train_data['sale_month'] = train_data['sale_date'].dt.month
train_data['sale_day'] = train_data['sale_date'].dt.day

# Repeat the same process for val_data
val_data['sale_date'] = pd.to_datetime(val_data['sale_date'])
val_data['sale_year'] = val_data['sale_date'].dt.year
val_data['sale_month'] = val_data['sale_date'].dt.month
val_data['sale_day'] = val_data['sale_date'].dt.day

# Drop the original 'sale_date' columns
train_data.drop('sale_date', axis=1, inplace=True)
val_data.drop('sale_date', axis=1, inplace=True)

# Fit model, seperating features and targets
model.fit(train_data.drop('sale_price', axis=1), train_data['sale_price'])

# Make predictions on the validation data
predicted_prices = model.predict(val_data)

# Create the base_result dataframe with house_id and predicted price
compare_result = pd.DataFrame({'house_id': val_data['house_id'], 'price': predicted_prices})

# Display the resulting dataframe
print(compare_result)

     house_id      price
0     1331375   81010.23
1     1630115  305520.52
2     1645745  399566.07
3     1336775  105839.73
4     1888274  265028.19
..        ...        ...
295   1986255  359711.91
296   1896276  382637.05
297   1758223  259560.68
298   1752010  175186.80
299   1651404  415871.50

[300 rows x 2 columns]


## **Checking the Root Mean Squared Erro (need to be below than 30000)**

In [86]:
import math
import numpy as np
y1 = base_result["price"]
y2 = compare_result["price"]

MSE = np.square(np.subtract(y1,y2)).mean() 

RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

18279.479019141927
